# Word Embeddings notícias de 2018

Os dados são notícias de 2018, coletadas dos principais jornais do Brasil:
* `Carta Capital`, 
* `Estadao`, 
* `Folha de São Paulo`, 
* `O Antagonista`, 
* `O Globo`, 
* `Veja`

Uma análise detalhada dos dados está disponível [aqui](https://pages.github.com/). Objetivo deste notebook é utilizar o modelo word2vec para gerar embeddings a partir dos textos dessas notícias. A arquitetura utilizada pelo modelo é a skip-gram, cada palavra é representada por um vetor de 300 dimensões.

In [1]:
# importing modules and setting log format
import re
import nltk
import gensim, logging
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
PUNCTUATION = u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ%]' # define news punctuation 

[nltk_data] Downloading package stopwords to /home/diogo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Definindo Lexicons e Funções

In [2]:
# Mapping words in lexicons
map_lexicons = {'a ponto':'a_ponto','ao menos ':'ao_menos ','ate mesmo ':'ate_mesmo ',
                'nao mais que ':'nao_mais_que ','nem mesmo ':'nem_mesmo ','no minimo ':'no_minimo ',
                'o unico ':'o_unico ','a unica ':'a_unica ','pelo menos ':'pelo_menos ',
                'quando menos ':'quando_menos ','quando muito ':'quando_muito ','a par disso ':'a_par_disso ',
                'e nao ':'e_nao ','em suma ':'em_suma ','mas tambem ': 'mas_tambem ','muito menos ':'muito_menos ',
                'nao so ':'nao_so ','ou mesmo ':'ou_mesmo ','por sinal ':'por_sinal ','com isso ':'com_isso ',
                'como consequencia ':'como_consequencia ','de modo que ':'de_modo_que ','deste modo ':'deste_modo ',
                'em decorrencia ':'em_decorrencia ','nesse sentido ':'nesse_sentido ','por causa ':'por_causa ',
                'por conseguinte ':'por_conseguinte ','por essa razao ':'por_essa_razao ','por isso ':'por_isso ',
                'sendo assim ':'sendo_assim ','ou entao ':'ou_entao ','ou mesmo ':'ou_mesmo ','como se ':'como_se ',
                'de um lado ':'de_um_lado ','por outro lado ':'por_outro_lado ','mais que ':'mais_que ',
                'menos que ':'menos_que ','desde que ':'desde_que ','do contrario ':'do_contrario ',
                'em lugar ':'em_lugar ','em vez ':'em_vez','no caso ':'no_caso ','se acaso ':'se_acaso ',
                'de certa forma ':'de_certa_forma ','desse modo ':'desse_modo ','em funcao ':'em_funcao ',
                'isso e ':'isso_e ','ja que ':'ja_que ','na medida que ':'na_medida_que ','nessa direcao ':'nessa_direcao ',
                'no intuito ':'no_intuito ','no mesmo sentido ':'no_mesmo_sentido ','ou seja ':'ou_seja ',
                'uma vez que ':'uma_vez_que ','tanto que ':'tanto_que ','visto que ':'visto_que ','ainda que ':'ainda_que ',
                'ao contrario ':'ao_contrario ','apesar de ':'apesar_de ','fora isso ':'fora_isso ','mesmo que ':'mesmo_que ',
                'nao obstante ':'nao_obstante ','nao fosse isso ':'nao_fosse_isso ','no entanto ':'no_entanto ',
                'para tanto ':'para_tanto ','pelo contrario ':'pelo_contrario ','por sua vez ':'por_sua_vez ','posto que ':'posto_que '
               }

In [3]:
# Convert word from text into lexicons
def word2lexicon(text):
    for k, v in map_lexicons.items():
        text = str(text).replace(k,v)
    return text

In [4]:
# function for processing sentences
def processSentences(text):
    stop_words = stopwords.words('portuguese') # load stop words
    text = re.sub(PUNCTUATION, ' ', str(text)) # remove punctuation from text
    text = str(text).split() # split sentences by words
    text = [word for word in text if word not in stop_words] # Remove stopwords
    return text

### Carregando Notícias

In [5]:
# load data
carta_capital = pd.read_csv("data/carta_capital.csv") 
estadao = pd.read_csv("data/estadao.csv")
folha = pd.read_csv("data/folha.csv") 
oantagonista = pd.read_csv("data/oantagonista.csv") 
oglobo = pd.read_csv("data/oglobo.csv") 
veja = pd.read_csv("data/veja.csv") 
# concat all news
news = pd.concat((carta_capital, folha, estadao, oantagonista, oglobo, veja), sort=False, ignore_index=True)

In [6]:
# processing news text
news['text'] = news['text'].apply(word2lexicon) 
news['text'] = news['text'].apply(processSentences)

### Treinando Word2Vec

In [7]:
# Train word2vec model - settings: approach skip-gram, size embeddings vectors 300 
model = gensim.models.Word2Vec(news['text'], workers=4, size=300, sg=1, window=5, min_count=5)
# Saving model
model.save('embeddings/news_w2v.bin')
# Saving embeddings
model.wv.save_word2vec_format("embeddings/news_vectors.bin")

2019-04-13 11:17:02,528 : INFO : collecting all words and their counts
2019-04-13 11:17:02,539 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-13 11:17:03,943 : INFO : PROGRESS: at sentence #10000, processed 4109020 words, keeping 118262 word types
2019-04-13 11:17:04,986 : INFO : PROGRESS: at sentence #20000, processed 7263023 words, keeping 148407 word types
2019-04-13 11:17:06,064 : INFO : PROGRESS: at sentence #30000, processed 10406672 words, keeping 172586 word types
2019-04-13 11:17:06,757 : INFO : PROGRESS: at sentence #40000, processed 12444180 words, keeping 181023 word types
2019-04-13 11:17:06,970 : INFO : PROGRESS: at sentence #50000, processed 13013149 words, keeping 184314 word types
2019-04-13 11:17:08,144 : INFO : PROGRESS: at sentence #60000, processed 16089672 words, keeping 221174 word types
2019-04-13 11:17:09,054 : INFO : PROGRESS: at sentence #70000, processed 18755851 words, keeping 230263 word types
2019-04-13 11:17:09,862 : I

2019-04-13 11:18:16,940 : INFO : EPOCH 1 - PROGRESS: at 4.50% examples, 76711 words/s, in_qsize 8, out_qsize 0
2019-04-13 11:18:18,066 : INFO : EPOCH 1 - PROGRESS: at 4.63% examples, 76725 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:18:19,276 : INFO : EPOCH 1 - PROGRESS: at 4.77% examples, 76988 words/s, in_qsize 8, out_qsize 0
2019-04-13 11:18:20,351 : INFO : EPOCH 1 - PROGRESS: at 4.87% examples, 76845 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:18:21,374 : INFO : EPOCH 1 - PROGRESS: at 4.98% examples, 76975 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:18:22,408 : INFO : EPOCH 1 - PROGRESS: at 5.08% examples, 77257 words/s, in_qsize 8, out_qsize 0
2019-04-13 11:18:23,744 : INFO : EPOCH 1 - PROGRESS: at 5.19% examples, 77091 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:18:24,798 : INFO : EPOCH 1 - PROGRESS: at 5.34% examples, 77520 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:18:25,971 : INFO : EPOCH 1 - PROGRESS: at 5.41% examples, 76750 words/s, in_qsize 7, out_qsize 0
2

2019-04-13 11:19:40,149 : INFO : EPOCH 1 - PROGRESS: at 14.19% examples, 74778 words/s, in_qsize 8, out_qsize 1
2019-04-13 11:19:41,806 : INFO : EPOCH 1 - PROGRESS: at 14.35% examples, 74570 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:19:42,870 : INFO : EPOCH 1 - PROGRESS: at 14.49% examples, 74555 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:19:43,921 : INFO : EPOCH 1 - PROGRESS: at 14.60% examples, 74533 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:19:45,037 : INFO : EPOCH 1 - PROGRESS: at 14.74% examples, 74561 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:19:46,211 : INFO : EPOCH 1 - PROGRESS: at 14.90% examples, 74623 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:19:47,215 : INFO : EPOCH 1 - PROGRESS: at 15.00% examples, 74569 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:19:48,467 : INFO : EPOCH 1 - PROGRESS: at 15.12% examples, 74452 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:19:49,540 : INFO : EPOCH 1 - PROGRESS: at 15.24% examples, 74433 words/s, in_qsize 7, out_

2019-04-13 11:21:02,210 : INFO : EPOCH 1 - PROGRESS: at 30.12% examples, 75423 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:21:03,260 : INFO : EPOCH 1 - PROGRESS: at 30.27% examples, 75463 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:21:04,513 : INFO : EPOCH 1 - PROGRESS: at 30.40% examples, 75378 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:21:05,613 : INFO : EPOCH 1 - PROGRESS: at 30.54% examples, 75355 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:21:06,890 : INFO : EPOCH 1 - PROGRESS: at 30.63% examples, 75218 words/s, in_qsize 8, out_qsize 0
2019-04-13 11:21:08,121 : INFO : EPOCH 1 - PROGRESS: at 30.75% examples, 75148 words/s, in_qsize 8, out_qsize 0
2019-04-13 11:21:09,136 : INFO : EPOCH 1 - PROGRESS: at 30.89% examples, 75198 words/s, in_qsize 7, out_qsize 1
2019-04-13 11:21:10,180 : INFO : EPOCH 1 - PROGRESS: at 31.05% examples, 75280 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:21:11,251 : INFO : EPOCH 1 - PROGRESS: at 31.14% examples, 75178 words/s, in_qsize 7, out_

2019-04-13 11:22:23,852 : INFO : EPOCH 1 - PROGRESS: at 40.89% examples, 75186 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:22:24,920 : INFO : EPOCH 1 - PROGRESS: at 40.91% examples, 75232 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:22:25,989 : INFO : EPOCH 1 - PROGRESS: at 40.92% examples, 75236 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:22:27,086 : INFO : EPOCH 1 - PROGRESS: at 40.93% examples, 75262 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:22:28,252 : INFO : EPOCH 1 - PROGRESS: at 40.96% examples, 75270 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:22:29,341 : INFO : EPOCH 1 - PROGRESS: at 41.08% examples, 75317 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:22:30,540 : INFO : EPOCH 1 - PROGRESS: at 41.19% examples, 75338 words/s, in_qsize 7, out_qsize 0
2019-04-13 11:22:31,686 : INFO : EPOCH 1 - PROGRESS: at 41.31% examples, 75375 words/s, in_qsize 8, out_qsize 0
2019-04-13 11:22:32,719 : INFO : EPOCH 1 - PROGRESS: at 41.42% examples, 75398 words/s, in_qsize 7, out_

...

2019-04-13 12:00:16,745 : INFO : EPOCH 5 - PROGRESS: at 31.03% examples, 84953 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:00:17,830 : INFO : EPOCH 5 - PROGRESS: at 31.19% examples, 84971 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:00:18,871 : INFO : EPOCH 5 - PROGRESS: at 31.32% examples, 84965 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:00:19,898 : INFO : EPOCH 5 - PROGRESS: at 31.46% examples, 84959 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:00:20,956 : INFO : EPOCH 5 - PROGRESS: at 31.62% examples, 84993 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:00:22,018 : INFO : EPOCH 5 - PROGRESS: at 31.77% examples, 84977 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:00:23,028 : INFO : EPOCH 5 - PROGRESS: at 31.93% examples, 84981 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:00:24,070 : INFO : EPOCH 5 - PROGRESS: at 32.07% examples, 84920 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:00:25,099 : INFO : EPOCH 5 - PROGRESS: at 32.43% examples, 84961 words/s, in_qsize 7, out_

2019-04-13 12:01:36,088 : INFO : EPOCH 5 - PROGRESS: at 42.29% examples, 85196 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:01:37,262 : INFO : EPOCH 5 - PROGRESS: at 42.43% examples, 85184 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:01:38,320 : INFO : EPOCH 5 - PROGRESS: at 42.56% examples, 85199 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:01:39,492 : INFO : EPOCH 5 - PROGRESS: at 42.66% examples, 85182 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:01:40,556 : INFO : EPOCH 5 - PROGRESS: at 42.77% examples, 85204 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:01:41,727 : INFO : EPOCH 5 - PROGRESS: at 42.91% examples, 85192 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:01:42,793 : INFO : EPOCH 5 - PROGRESS: at 43.05% examples, 85206 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:01:43,953 : INFO : EPOCH 5 - PROGRESS: at 43.19% examples, 85198 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:01:45,039 : INFO : EPOCH 5 - PROGRESS: at 43.33% examples, 85213 words/s, in_qsize 7, out_

2019-04-13 12:02:55,824 : INFO : EPOCH 5 - PROGRESS: at 67.21% examples, 85418 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:02:56,917 : INFO : EPOCH 5 - PROGRESS: at 67.38% examples, 85425 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:02:57,993 : INFO : EPOCH 5 - PROGRESS: at 67.53% examples, 85432 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:02:59,156 : INFO : EPOCH 5 - PROGRESS: at 67.69% examples, 85422 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:03:00,234 : INFO : EPOCH 5 - PROGRESS: at 67.85% examples, 85431 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:03:01,431 : INFO : EPOCH 5 - PROGRESS: at 68.02% examples, 85416 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:03:02,444 : INFO : EPOCH 5 - PROGRESS: at 68.18% examples, 85440 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:03:03,672 : INFO : EPOCH 5 - PROGRESS: at 68.34% examples, 85417 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:03:04,674 : INFO : EPOCH 5 - PROGRESS: at 68.51% examples, 85445 words/s, in_qsize 7, out_

2019-04-13 12:04:17,440 : INFO : EPOCH 5 - PROGRESS: at 78.94% examples, 85434 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:04:18,602 : INFO : EPOCH 5 - PROGRESS: at 79.09% examples, 85426 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:04:19,612 : INFO : EPOCH 5 - PROGRESS: at 79.26% examples, 85447 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:04:20,841 : INFO : EPOCH 5 - PROGRESS: at 79.42% examples, 85425 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:04:21,935 : INFO : EPOCH 5 - PROGRESS: at 79.59% examples, 85452 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:04:23,075 : INFO : EPOCH 5 - PROGRESS: at 79.74% examples, 85427 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:04:24,158 : INFO : EPOCH 5 - PROGRESS: at 79.88% examples, 85413 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:04:25,326 : INFO : EPOCH 5 - PROGRESS: at 80.03% examples, 85406 words/s, in_qsize 6, out_qsize 1
2019-04-13 12:04:26,482 : INFO : EPOCH 5 - PROGRESS: at 80.19% examples, 85401 words/s, in_qsize 8, out_

2019-04-13 12:05:39,252 : INFO : EPOCH 5 - PROGRESS: at 95.16% examples, 85379 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:05:40,384 : INFO : EPOCH 5 - PROGRESS: at 95.33% examples, 85360 words/s, in_qsize 8, out_qsize 0
2019-04-13 12:05:41,441 : INFO : EPOCH 5 - PROGRESS: at 95.56% examples, 85391 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:05:42,644 : INFO : EPOCH 5 - PROGRESS: at 95.75% examples, 85362 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:05:43,658 : INFO : EPOCH 5 - PROGRESS: at 96.03% examples, 85400 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:05:44,900 : INFO : EPOCH 5 - PROGRESS: at 96.26% examples, 85364 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:05:46,255 : INFO : EPOCH 5 - PROGRESS: at 96.58% examples, 85366 words/s, in_qsize 7, out_qsize 0
2019-04-13 12:05:47,611 : INFO : EPOCH 5 - PROGRESS: at 96.93% examples, 85368 words/s, in_qsize 6, out_qsize 2
2019-04-13 12:05:48,649 : INFO : EPOCH 5 - PROGRESS: at 97.10% examples, 85406 words/s, in_qsize 7, out_

In [10]:
print(model.wv.most_similar(positive=[u'pontes'], negative=[u'presidente']))

[('rousseff', 0.5715487003326416), ('impeachment', 0.33618271350860596), ('roussef', 0.3163612186908722), ('pedaladas', 0.3028866648674011), ('cassada', 0.2818000912666321), ('gerentona', 0.276383638381958), ('petrolão', 0.24935714900493622), ('desempregados', 0.2331179678440094), ('janete', 0.2272392213344574), ('desgovernos', 0.22592443227767944)]


In [3]:
from gensim.models import Word2Vec
new_model = Word2Vec.load('embeddings/news_w2v.bin')

2019-04-30 13:12:20,064 : INFO : loading Word2Vec object from embeddings/news_w2v.bin
2019-04-30 13:12:20,777 : INFO : loading wv recursively from embeddings/news_w2v.bin.wv.* with mmap=None
2019-04-30 13:12:20,778 : INFO : loading vectors from embeddings/news_w2v.bin.wv.vectors.npy with mmap=None
2019-04-30 13:12:22,499 : INFO : setting ignored attribute vectors_norm to None
2019-04-30 13:12:22,502 : INFO : loading vocabulary recursively from embeddings/news_w2v.bin.vocabulary.* with mmap=None
2019-04-30 13:12:22,503 : INFO : loading trainables recursively from embeddings/news_w2v.bin.trainables.* with mmap=None
2019-04-30 13:12:22,504 : INFO : loading syn1neg from embeddings/news_w2v.bin.trainables.syn1neg.npy with mmap=None
2019-04-30 13:12:24,017 : INFO : setting ignored attribute cum_table to None
2019-04-30 13:12:24,021 : INFO : loaded embeddings/news_w2v.bin


In [31]:
print(new_model.wv.most_similar(positive=[u'guedes']))

[('superministro', 0.7084858417510986), ('ultraliberal', 0.5786339044570923), ('superministérios', 0.5777881145477295), ('superpasta', 0.5672988891601562), ('delegará', 0.5668069124221802), ('onyx', 0.5664126873016357), ('ministeriável', 0.5586702227592468), ('guru', 0.5561445355415344), ('superministros', 0.5527079105377197), ('superministério', 0.5492182970046997)]
